# Day10_1: A/B 테스트 분석 (A/B Test Analysis)

## 📚 학습 목표

**Part 1: 기초**
1. A/B 테스트의 기본 개념과 원리 이해하기
2. A/B 그룹별 기초 통계량 계산하기
3. 전환율(Conversion Rate) 계산과 비교하기
4. Plotly로 그룹 비교 시각화하기
5. 통계적 유의성 판단하기

**Part 2: 심화**
1. 비율 검정(z-test)으로 A/B 그룹 비교하기
2. 신뢰구간(Confidence Interval) 계산하기
3. 효과 크기(Effect Size) 측정하기
4. 세그먼트별 분석과 의사결정하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| A/B 테스트 | 데이터 기반 의사결정 | "어떤 버튼 색상이 전환율을 높이는가?" |
| 전환율 계산 | KPI 측정 | 방문자 중 구매자 비율 |
| 비율 검정 | 통계적 검증 | 전환율 차이가 우연인지 실제 효과인지 |
| 신뢰구간 | 불확실성 정량화 | "전환율이 3~5% 사이일 것" |
| 효과 크기 | 실용적 중요성 | 통계적 유의성보다 실질적 효과 |

**분석가 관점**: A/B 테스트는 제품 개선, 마케팅 캔페인, UX 변경의 효과를 과학적으로 검증하는 핵심 방법론입니다!

---

# Part 1: 기초

---

## 1.1 A/B 테스트의 기본 개념

### A/B 테스트란?

**A/B 테스트(A/B Testing)**는 두 가지 버전(A와 B)을 무작위로 사용자에게 노출하여 어떤 것이 더 효과적인지 비교하는 실험 방법입니다.

### 핵심 용어

- **통제그룹(Control, A)**: 기존 버전
- **실험그룹(Treatment, B)**: 변경된 새 버전
- **전환율(Conversion Rate)**: 목표 행동을 수행한 비율
- **통계적 유의성**: 차이가 우연이 아닌지 판단

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("✅ 라이브러리 임포트 완료")

In [ ]:
# A/B 테스트 데이터 로드 (2주간 버튼 색상 테스트)
df = pd.read_csv('datasets/ab_test_button_color.csv')

print("A/B 테스트 데이터 개요:")
print(f"전체 샘플 수: {len(df):,}개")
print(f"\n칼럼: {list(df.columns)}")
print(f"\n데이터 타입:\n{df.dtypes}")

In [ ]:
# 데이터 샘플 확인
print("데이터 샘플 (5행):")
df.head()

### 데이터 설명

| 칼럼 | 설명 |
|--------|------|
| `user_id` | 사용자 고유 ID |
| `group` | A(통제: 녹색 버튼) / B(실험: 주황색 버튼) |
| `visited` | 페이지 방문 여부 (1=방문) |
| `converted` | 전환 여부 (1=구매 완료) |
| `session_duration` | 세션 지속시간(초) |
| `device` | 디바이스 유형 |
| `timestamp` | 방문 시간 |

---

## 1.2 A/B 그룹별 기초 통계량

### 그룹별 샘플 수 확인

In [ ]:
# 그룹별 샘플 수
group_counts = df['group'].value_counts().sort_index()
print("그룹별 샘플 수:")
print(group_counts)
print(f"\n전체: {group_counts.sum()}개")

In [ ]:
# 그룹별 기초 통계량
group_stats = df.groupby('group').agg({
    'user_id': 'count',
    'converted': ['sum', 'mean'],
    'session_duration': ['mean', 'std']
}).round(2)

# 칼럼명 정리
group_stats.columns = ['샘플수', '전환수', '전환율', '평균체류시간', '체류시간_std']
print("그룹별 기초 통계량:")
group_stats

### 💡 실무 예시: 샘플 크기 균형 확인

A/B 테스트에서는 두 그룹의 샘플 크기가 비슷해야 공정한 비교가 가능합니다.

In [ ]:
# 샘플 크기 균형 확인
n_a = len(df[df['group'] == 'A'])
n_b = len(df[df['group'] == 'B'])

balance_ratio = min(n_a, n_b) / max(n_a, n_b) * 100

print(f"A그룹: {n_a}개")
print(f"B그룹: {n_b}개")
print(f"균형 비율: {balance_ratio:.1f}%")

if balance_ratio >= 90:
    print("✅ 샘플 크기가 잘 균형잡혀 있습니다.")
else:
    print("⚠️ 샘플 크기 불균형이 있습니다. 결과 해석 시 주의가 필요합니다.")

---

## 1.3 전환율(Conversion Rate) 계산

### 전환율 공식

$$\text{Conversion Rate} = \frac{\text{Number of Conversions}}{\text{Number of Visitors}} \times 100$$

In [ ]:
# 그룹별 전환율 계산
group_a = df[df['group'] == 'A']
group_b = df[df['group'] == 'B']

# 전환 수
conversions_a = group_a['converted'].sum()
conversions_b = group_b['converted'].sum()

# 전환율
cr_a = conversions_a / len(group_a) * 100
cr_b = conversions_b / len(group_b) * 100

print("=" * 50)
print("전환율 분석 결과")
print("=" * 50)
print(f"\nA그룹 (녹색 버튼):")
print(f"  방문자: {len(group_a)}명")
print(f"  전환 수: {conversions_a}건")
print(f"  전환율: {cr_a:.2f}%")

print(f"\nB그룹 (주황색 버튼):")
print(f"  방문자: {len(group_b)}명")
print(f"  전환 수: {conversions_b}건")
print(f"  전환율: {cr_b:.2f}%")

print(f"\n전환율 차이: {cr_b - cr_a:.2f}%p")
print(f"상대적 개선율: {(cr_b - cr_a) / cr_a * 100:.1f}%")

### 💡 실무 예시: 비즈니스 임팩트 계산

전환율 상승이 비즈니스에 얼마나 큰 영향을 미치는지 계산해봅시다.

In [ ]:
# 비즈니스 임팩트 예시
monthly_visitors = 100000  # 월 방문자
avg_order_value = 50000    # 평균 주문 금액 (50,000원)

# 기존 (A) 예상 매출
expected_conversions_a = monthly_visitors * (cr_a / 100)
expected_revenue_a = expected_conversions_a * avg_order_value

# 신규 (B) 예상 매출
expected_conversions_b = monthly_visitors * (cr_b / 100)
expected_revenue_b = expected_conversions_b * avg_order_value

# 추가 매출
additional_revenue = expected_revenue_b - expected_revenue_a

print("💰 비즈니스 임팩트 분석")
print("=" * 50)
print(f"\n월 방문자 수: {monthly_visitors:,}명")
print(f"평균 주문 금액: {avg_order_value:,}원")
print(f"\nA그룹 예상 월 매출: {expected_revenue_a:,.0f}원")
print(f"B그룹 예상 월 매출: {expected_revenue_b:,.0f}원")
print(f"\n🚀 B그룹 적용 시 추가 월 매출: {additional_revenue:,.0f}원")
print(f"   연간 추가 매출: {additional_revenue * 12:,.0f}원")

---

## 1.4 Plotly로 그룹 비교 시각화

### 전환율 비교 바 차트

In [ ]:
# 그룹별 전환율 데이터 준비
conversion_data = pd.DataFrame({
    '그룹': ['A (녹색 버튼)', 'B (주황색 버튼)'],
    '전환율': [cr_a, cr_b],
    '방문자': [len(group_a), len(group_b)],
    '전환수': [conversions_a, conversions_b]
})

# 바 차트 생성
fig = px.bar(
    conversion_data,
    x='그룹',
    y='전환율',
    color='그룹',
    text=conversion_data['전환율'].apply(lambda x: f'{x:.2f}%'),
    title='A/B 테스트: 그룹별 전환율 비교',
    color_discrete_map={'A (녹색 버튼)': '#2ecc71', 'B (주황색 버튼)': '#e67e22'},
    width=600,
    height=450
)

fig.update_traces(textposition='outside', textfont_size=14)
fig.update_layout(
    yaxis_title='전환율 (%)',
    xaxis_title='',
    showlegend=False,
    font=dict(size=12)
)

fig.show()

### 전환 깨때기(연도 차트) 시각화

In [ ]:
# 전환 깨때기 데이터 준비
funnel_data = pd.DataFrame({
    '단계': ['방문', '전환'] * 2,
    '그룹': ['A그룹', 'A그룹', 'B그룹', 'B그룹'],
    '수': [len(group_a), conversions_a, len(group_b), conversions_b]
})

fig = go.Figure()

# A그룹 퍼널
fig.add_trace(go.Funnel(
    name='A그룹 (녹색)',
    y=['방문', '전환'],
    x=[len(group_a), conversions_a],
    textinfo='value+percent previous',
    marker=dict(color=['#27ae60', '#27ae60']),
    textfont=dict(size=13)
))

# B그룹 퍼널
fig.add_trace(go.Funnel(
    name='B그룹 (주황)',
    y=['방문', '전환'],
    x=[len(group_b), conversions_b],
    textinfo='value+percent previous',
    marker=dict(color=['#e67e22', '#e67e22']),
    textfont=dict(size=13)
))

fig.update_layout(
    title='A/B 그룹별 전환 퍼널',
    width=700,
    height=400
)

fig.show()

### 일별 전환율 추이 시각화

In [ ]:
# 날짜 처리
df['date'] = pd.to_datetime(df['timestamp']).dt.date

# 일별 전환율 계산
daily_cr = df.groupby(['date', 'group']).agg({
    'converted': ['sum', 'count']
}).reset_index()
daily_cr.columns = ['date', 'group', 'conversions', 'visitors']
daily_cr['conversion_rate'] = daily_cr['conversions'] / daily_cr['visitors'] * 100

# 라인 차트
fig = px.line(
    daily_cr,
    x='date',
    y='conversion_rate',
    color='group',
    markers=True,
    title='일별 전환율 추이',
    labels={'date': '날짜', 'conversion_rate': '전환율 (%)', 'group': '그룹'},
    color_discrete_map={'A': '#2ecc71', 'B': '#e67e22'},
    width=800,
    height=400
)

fig.update_layout(font=dict(size=12))
fig.show()

---

## 1.5 통계적 유의성 판단

### 왜 통계적 검정이 필요한가?

전환율이 다르더라도, 그 차이가:
- **우연**일 수 있음 (샘플링 변동)
- **실제 효과**일 수 있음 (버튼 색상의 영향)

통계적 검정으로 이를 판단합니다!

In [ ]:
# 기초적인 t-test (Part 2에서 더 정확한 z-test 사용)
# 전환 여부는 0/1 값이므로 t-test로 평균 차이 검정 가능

t_stat, p_value = stats.ttest_ind(
    group_a['converted'],
    group_b['converted']
)

alpha = 0.05  # 유의수준

print("통계적 유의성 검정 (기초)")
print("=" * 50)
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"유의수준 (α): {alpha}")
print()

if p_value < alpha:
    print(f"✅ p-value ({p_value:.4f}) < α ({alpha})")
    print("→ 귀무가설 기각: A와 B 그룹의 전환율에 통계적으로 유의한 차이가 있습니다!")
else:
    print(f"❌ p-value ({p_value:.4f}) >= α ({alpha})")
    print("→ 귀무가설 채택: 전환율 차이가 통계적으로 유의하지 않습니다.")

---

# Part 2: 심화

---

## 2.1 비율 검정(z-test)으로 A/B 그룹 비교

### 비율 검정이란?

전환율처럼 **비율(비율)** 데이터를 비교할 때는 **z-test for proportions**가 적합합니다.

### 가설 설정

- **H₀ (귀무가설)**: p_A = p_B (두 그룹의 전환율이 같다)
- **H₁ (대립가설)**: p_A ≠ p_B (두 그룹의 전환율이 다르다)

In [ ]:
# 비율 검정 (Two-sample z-test for proportions)
from statsmodels.stats.proportion import proportions_ztest

# 전환 수와 샘플 수
count = np.array([conversions_a, conversions_b])
nobs = np.array([len(group_a), len(group_b)])

# z-test 수행
z_stat, p_value_z = proportions_ztest(count, nobs, alternative='two-sided')

print("비율 검정 (Two-sample z-test)")
print("=" * 50)
print(f"\nA그룹 전환율: {cr_a:.2f}%")
print(f"B그룹 전환율: {cr_b:.2f}%")
print(f"\nz-statistic: {z_stat:.4f}")
print(f"p-value: {p_value_z:.6f}")

print("\n" + "=" * 50)
if p_value_z < 0.05:
    print("✅ 결론: B그룹(주황색 버튼)이 A그룹(녹색 버튼)보다")
    print(f"   통계적으로 유의하게 높은 전환율을 보입니다!")
    print(f"   (p-value = {p_value_z:.6f} < 0.05)")
else:
    print("❌ 결론: 두 그룹 간 전환율 차이가 통계적으로 유의하지 않습니다.")
print("=" * 50)

### 💡 실무 예시: 단측 검정 vs 양측 검정

- **양측 검정(two-sided)**: A와 B가 다른지 검정
- **단측 검정(one-sided)**: B가 A보다 더 큰지 검정

In [ ]:
# 단측 검정: B > A 인지 검정 (B가 더 효과적인지)
z_stat_one, p_value_one = proportions_ztest(count, nobs, alternative='larger')

print("단측 검정 (H₁: B그룹 > A그룹)")
print("=" * 50)
print(f"z-statistic: {z_stat_one:.4f}")
print(f"p-value: {p_value_one:.6f}")

if p_value_one < 0.05:
    print(f"\n✅ B그룹이 A그룹보다 통계적으로 유의하게 높은 전환율을 보입니다.")
else:
    print(f"\n❌ B그룹이 A그룹보다 높다는 증거가 부족합니다.")

---

## 2.2 신뢰구간(Confidence Interval) 계산

### 신뢰구간이란?

신뢰구간은 **모수(모집단의 실제 값)가 있을 것으로 예상되는 범위**입니다.

- **95% 신뢰구간**: 100번 실험하면 95번은 이 범위 안에 실제 값이 있음

In [ ]:
from statsmodels.stats.proportion import proportion_confint

# 각 그룹의 95% 신뢰구간 계산
ci_a = proportion_confint(conversions_a, len(group_a), alpha=0.05, method='wilson')
ci_b = proportion_confint(conversions_b, len(group_b), alpha=0.05, method='wilson')

print("전환율 95% 신뢰구간")
print("=" * 50)
print(f"\nA그룹: {cr_a:.2f}% [{ci_a[0]*100:.2f}%, {ci_a[1]*100:.2f}%]")
print(f"B그룹: {cr_b:.2f}% [{ci_b[0]*100:.2f}%, {ci_b[1]*100:.2f}%]")

### 신뢰구간 시각화

In [ ]:
# 신뢰구간 시각화
fig = go.Figure()

# A그룹
fig.add_trace(go.Scatter(
    x=['A그룹 (녹색)'],
    y=[cr_a],
    error_y=dict(
        type='data',
        symmetric=False,
        array=[ci_a[1]*100 - cr_a],
        arrayminus=[cr_a - ci_a[0]*100]
    ),
    mode='markers',
    marker=dict(size=15, color='#2ecc71'),
    name='A그룹'
))

# B그룹
fig.add_trace(go.Scatter(
    x=['B그룹 (주황)'],
    y=[cr_b],
    error_y=dict(
        type='data',
        symmetric=False,
        array=[ci_b[1]*100 - cr_b],
        arrayminus=[cr_b - ci_b[0]*100]
    ),
    mode='markers',
    marker=dict(size=15, color='#e67e22'),
    name='B그룹'
))

fig.update_layout(
    title='전환율 95% 신뢰구간 비교',
    yaxis_title='전환율 (%)',
    xaxis_title='',
    showlegend=True,
    width=600,
    height=450,
    font=dict(size=12)
)

fig.show()

# 신뢰구간 겹침 여부 확인
if ci_a[1] < ci_b[0]:
    print("\n✅ 신뢰구간이 겹치지 않습니다 → B그룹이 확실히 더 높습니다!")
elif ci_b[1] < ci_a[0]:
    print("\n✅ 신뢰구간이 겹치지 않습니다 → A그룹이 확실히 더 높습니다!")
else:
    print("\n⚠️ 신뢰구간이 겹칩니다 → 차이의 확실성이 낮습니다.")

### 전환율 차이의 신뢰구간

In [ ]:
# 전환율 차이의 신뢰구간 계산
from statsmodels.stats.proportion import confint_proportions_2indep

ci_diff = confint_proportions_2indep(
    conversions_b, len(group_b),  # B그룹 (비교 대상)
    conversions_a, len(group_a),  # A그룹 (기준)
    method='wald',
    alpha=0.05
)

diff = cr_b - cr_a

print("전환율 차이 (B - A)의 95% 신뢰구간")
print("=" * 50)
print(f"\n차이: {diff:.2f}%p")
print(f"95% CI: [{ci_diff[0]*100:.2f}%p, {ci_diff[1]*100:.2f}%p]")

if ci_diff[0] > 0:
    print("\n✅ 신뢰구간 하한이 0보다 큽니다 → B그룹이 확실히 더 높습니다!")
elif ci_diff[1] < 0:
    print("\n✅ 신뢰구간 상한이 0보다 작습니다 → A그룹이 확실히 더 높습니다!")
else:
    print("\n⚠️ 신뢰구간이 0을 포함합니다 → 차이가 통계적으로 유의하지 않습니다.")

---

## 2.3 효과 크기(Effect Size) 측정

### 효과 크기란?

**효과 크기**는 통계적 유의성과 별개로, **실제로 얼마나 큰 차이인지**를 나타냅니다.

### Cohen's h (비율 차이의 효과 크기)

- |h| < 0.2: 작은 효과
- 0.2 ≤ |h| < 0.5: 중간 효과
- |h| ≥ 0.5: 큰 효과

In [ ]:
# Cohen's h 계산 (비율 차이의 효과 크기)
import math

def cohens_h(p1, p2):
    """Cohen's h 효과 크기 계산"""
    phi1 = 2 * math.asin(math.sqrt(p1))
    phi2 = 2 * math.asin(math.sqrt(p2))
    return abs(phi1 - phi2)

p_a = cr_a / 100
p_b = cr_b / 100

h = cohens_h(p_a, p_b)

print("효과 크기 (Cohen's h)")
print("=" * 50)
print(f"\nCohen's h: {h:.4f}")

if h < 0.2:
    effect = "작은 효과 (Small)"
elif h < 0.5:
    effect = "중간 효과 (Medium)"
else:
    effect = "큰 효과 (Large)"

print(f"해석: {effect}")
print("\n해석 기준:")
print("  |h| < 0.2  : 작은 효과")
print("  0.2 ≤ |h| < 0.5: 중간 효과")
print("  |h| ≥ 0.5 : 큰 효과")

### 💡 실무 예시: 통계적 유의성 vs 실무적 중요성

- **통계적 유의성**: 샘플 크기가 크면 아주 작은 차이도 유의할 수 있음
- **실무적 중요성**: 그 차이가 비즈니스에 의미 있는지는 별개의 문제

In [ ]:
# 상대적 개선율 (Relative Lift)
relative_lift = (cr_b - cr_a) / cr_a * 100

# 절대적 개선율 (Absolute Lift)
absolute_lift = cr_b - cr_a

print("실무적 중요성 평가")
print("=" * 50)
print(f"\n절대적 개선율: {absolute_lift:.2f}%p")
print(f"상대적 개선율: {relative_lift:.1f}%")
print(f"\n효과 크기 (Cohen's h): {h:.4f} ({effect})")

# 비즈니스 임팩트 재계산
print(f"\n월 추가 매출 (예상): {additional_revenue:,.0f}원")

---

## 2.4 세그먼트별 분석과 의사결정

### 디바이스별 전환율 비교

In [ ]:
# 디바이스별 전환율 계산
device_cr = df.groupby(['device', 'group']).agg({
    'converted': ['sum', 'count']
}).reset_index()
device_cr.columns = ['device', 'group', 'conversions', 'visitors']
device_cr['conversion_rate'] = device_cr['conversions'] / device_cr['visitors'] * 100

print("디바이스별 전환율:")
device_pivot = device_cr.pivot(index='device', columns='group', values='conversion_rate').round(2)
device_pivot['diff'] = device_pivot['B'] - device_pivot['A']
device_pivot.columns = ['A그룹 (%)', 'B그룹 (%)', '차이 (%p)']
device_pivot

In [ ]:
# 디바이스별 전환율 시각화
fig = px.bar(
    device_cr,
    x='device',
    y='conversion_rate',
    color='group',
    barmode='group',
    text=device_cr['conversion_rate'].apply(lambda x: f'{x:.1f}%'),
    title='디바이스별 A/B 전환율 비교',
    labels={'device': '디바이스', 'conversion_rate': '전환율 (%)', 'group': '그룹'},
    color_discrete_map={'A': '#2ecc71', 'B': '#e67e22'},
    width=700,
    height=450
)

fig.update_traces(textposition='outside')
fig.update_layout(font=dict(size=12))
fig.show()

### 최종 의사결정 리포트

In [ ]:
# 최종 리포트 생성
print("=" * 70)
print("                A/B 테스트 최종 분석 리포트")
print("=" * 70)

print("\n📊 실험 개요")
print("-" * 70)
print(f"실험 기간: 2025-01-01 ~ 2025-01-18 (18일)")
print(f"실험 대상: 결제 버튼 색상 (녹색 vs 주황색)")
print(f"총 샘플: {len(df):,}개 (A: {len(group_a)}, B: {len(group_b)})")

print("\n📈 주요 결과")
print("-" * 70)
print(f"A그룹 전환율: {cr_a:.2f}%")
print(f"B그룹 전환율: {cr_b:.2f}%")
print(f"절대적 개선: {absolute_lift:.2f}%p")
print(f"상대적 개선: {relative_lift:.1f}%")

print("\n🔬 통계적 검증")
print("-" * 70)
print(f"p-value: {p_value_z:.6f}")
print(f"통계적 유의성: {'Yes' if p_value_z < 0.05 else 'No'} (α=0.05)")
print(f"효과 크기 (Cohen's h): {h:.4f} ({effect})")
print(f"95% CI: [{ci_diff[0]*100:.2f}%p, {ci_diff[1]*100:.2f}%p]")

print("\n💰 비즈니스 임팩트 (월 방문자 100,000명 기준)")
print("-" * 70)
print(f"월 추가 전환: {expected_conversions_b - expected_conversions_a:,.0f}건")
print(f"월 추가 매출: {additional_revenue:,.0f}원")
print(f"연간 추가 매출: {additional_revenue * 12:,.0f}원")

print("\n🎯 권장 사항")
print("=" * 70)
if p_value_z < 0.05 and cr_b > cr_a:
    print("✅ B그룹(주황색 버튼)을 전체 사용자에게 적용하시기 바랍니다.")
    print("   - 통계적으로 유의한 전환율 개선 확인")
    print("   - 연간 상당한 매출 증가 기대")
else:
    print("⚠️ 추가 테스트가 필요합니다.")
    print("   - 샘플 크기를 늘리거나")
    print("   - 다른 요소(CTA 문구, 버튼 위치 등)를 테스트하세요")
print("=" * 70)

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 전환율 계산하기 ⭐

**문제**: 다음 데이터에서 A그룹과 B그룹의 전환율을 계산하세요.

```python
# 방문자 수
visitors_a = 500
visitors_b = 500

# 전환 수
conversions_a = 45
conversions_b = 60
```

**기대 결과**: A그룹 전환율, B그룹 전환율, 차이

In [ ]:
visitors_a = 500
visitors_b = 500
conversions_a = 45
conversions_b = 60

# 여기에 코드를 작성하세요


### Q2. 그룹별 기초 통계량 계산 ⭐

**문제**: 이메일 A/B 테스트 데이터를 로드하여 그룹별 오픈율과 클릭율을 계산하세요.

```python
email_df = pd.read_csv('datasets/ab_test_email_subject.csv')
```

**기대 결과**: 그룹별 오픈율(오픈/발송), 클릭율(클릭/오픈)

In [ ]:
email_df = pd.read_csv('datasets/ab_test_email_subject.csv')

# 여기에 코드를 작성하세요


### Q3. Plotly 바 차트로 전환율 비교 ⭐⭐

**문제**: 다음 데이터로 Plotly 바 차트를 생성하세요.

```python
campaign_data = pd.DataFrame({
    '캔페인': ['A (10% 할인)', 'B (20% 할인)', 'C (1+1 행사)'],
    '전환율': [3.2, 4.5, 5.1]
})
```

**요구사항**: 막대 위에 전환율 표시, 적절한 제목

In [ ]:
campaign_data = pd.DataFrame({
    '캔페인': ['A (10% 할인)', 'B (20% 할인)', 'C (1+1 행사)'],
    '전환율': [3.2, 4.5, 5.1]
})

# 여기에 코드를 작성하세요


### Q4. t-test로 전환율 차이 검정 ⭐⭐

**문제**: 다음 데이터로 두 그룹의 전환율 차이가 통계적으로 유의한지 검정하세요.

```python
# 각 사용자의 전환 여부 (0 또는 1)
group_a_conv = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]  # 전환율 30%
group_b_conv = [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]  # 전환율 60%
```

**기대 결과**: t-statistic, p-value, 결론

In [ ]:
group_a_conv = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
group_b_conv = [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]

# 여기에 코드를 작성하세요


### Q5. p-value 해석 ⭐⭐

**문제**: 다음 p-value 값들에 대해 귀무가설 기각 여부를 판단하고 해석하세요.

```python
p_values = [0.001, 0.03, 0.05, 0.12, 0.48]
alpha = 0.05
```

**기대 결과**: 각 p-value에 대한 해석

In [ ]:
p_values = [0.001, 0.03, 0.05, 0.12, 0.48]
alpha = 0.05

# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 비율 검정 (z-test) 수행 ⭐⭐⭐

**문제**: 다음 데이터로 z-test를 수행하여 두 그룹의 전환율 차이를 검정하세요.

```python
# 각 그룹 전환 수와 총 샘플
conversions = np.array([85, 105])  # A그룹 85건, B그룹 105건 전환
samples = np.array([1000, 1000])   # 각 1000명씩
```

**기대 결과**: z-statistic, p-value, 결론

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

conversions = np.array([85, 105])
samples = np.array([1000, 1000])

# 여기에 코드를 작성하세요


### Q7. 신뢰구간 계산하기 ⭐⭐⭐⭐

**문제**: 전환율 12% (120/1000)의 95% 신뢰구간을 계산하세요.

```python
conversions = 120
total = 1000
```

**기대 결과**: 전환율과 95% 신뢰구간 [하한, 상한]

In [ ]:
from statsmodels.stats.proportion import proportion_confint

conversions = 120
total = 1000

# 여기에 코드를 작성하세요


### Q8. 신뢰구간 시각화 ⭐⭐⭐⭐

**문제**: 3개 캔페인의 전환율과 신뢰구간을 오차 바 차트로 시각화하세요.

```python
campaigns = pd.DataFrame({
    '캔페인': ['A', 'B', 'C'],
    '전환율': [4.5, 5.2, 4.8],
    'CI_lower': [3.8, 4.6, 4.1],
    'CI_upper': [5.2, 5.8, 5.5]
})
```

**요구사항**: `go.Scatter`로 오차 바 표시

In [ ]:
campaigns = pd.DataFrame({
    '캔페인': ['A', 'B', 'C'],
    '전환율': [4.5, 5.2, 4.8],
    'CI_lower': [3.8, 4.6, 4.1],
    'CI_upper': [5.2, 5.8, 5.5]
})

# 여기에 코드를 작성하세요


### Q9. Cohen's h 효과 크기 계산 ⭐⭐⭐⭐⭐

**문제**: 다음 두 그룹의 전환율 차이에 대한 Cohen's h를 계산하고 해석하세요.

```python
p_control = 0.08  # 통제그룹 전환율 8%
p_treatment = 0.12  # 실험그룹 전환율 12%
```

**기대 결과**: Cohen's h 값과 효과 크기 해석

In [ ]:
import math

p_control = 0.08
p_treatment = 0.12

# 여기에 코드를 작성하세요


### Q10. 종합 A/B 테스트 분석 리포트 ⭐⭐⭐⭐⭐

**문제**: 이메일 마케팅 A/B 테스트 데이터를 완전히 분석하세요.

```python
email_df = pd.read_csv('datasets/ab_test_email_subject.csv')
```

**요구사항**:
1. 그룹별 오픈율, 클릭율 계산
2. z-test로 오픈율 차이 검정
3. 95% 신뢰구간 계산
4. Cohen's h 효과 크기 계산
5. 세그먼트별(user_segment) 분석
6. 최종 권고사항 작성

In [ ]:
email_df = pd.read_csv('datasets/ab_test_email_subject.csv')

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 설명 | 활용 |
|------|------|------|
| A/B 테스트 | 두 버전 비교 실험 | 제품/마케팅 개선 효과 검증 |
| 전환율 | 전환수/방문자 x 100 | 핵심 KPI 측정 |
| p-value | 우연히 차이가 나타날 확률 | < 0.05면 통계적 유의 |
| 유의수준 | 기각 기준 (보통 0.05) | 실험 엄격도 설정 |

### Part 2: 심화 핵심 요약

| 개념 | 공식/함수 | 활용 |
|------|----------|------|
| z-test (비율) | `proportions_ztest(count, nobs)` | 전환율 차이 검정 |
| 신뢰구간 | `proportion_confint()` | 모수 범위 추정 |
| Cohen's h | 2*arcsin(√p) 차이 | 효과 크기 평가 |
| 세그먼트 분석 | groupby + 검정 | 하위그룹별 효과 확인 |

### 💡 실무 팁

1. **샘플 크기 확인**: 최소 1,000명 이상 권장, 귀피 테스트 권장
2. **실험 기간**: 요일 효과를 반영하려면 최소 1-2주
3. **단측 vs 양측**: 방향이 확실하면 단측, 불확실하면 양측 검정
4. **효과 크기**: 통계적 유의성보다 실무적 중요성 고려
5. **세그먼트 분석**: 전체 결과와 하위그룹 결과가 다를 수 있음
6. **비즈니스 임팩트**: 최종 의사결정은 매출/비용 기준으로

### 📈 A/B 테스트 의사결정 플로우

```
실험 설계
    ↓
데이터 수집 (최소 1-2주)
    ↓
샘플 크기 확인 (각 그룹 1000+)
    ↓
전환율 계산
    ↓
z-test 수행 → p-value < 0.05?
    ↓
신뢰구간 겹침 확인
    ↓
효과 크기 평가
    ↓
세그먼트별 분석
    ↓
비즈니스 임팩트 계산
    ↓
최종 의사결정
```